This is the original model, run in Python 2

# Load data

In [1]:
import os
import argparse
import pandas as pd
import numpy as np
import pickle

In [2]:
from keras.models import Sequential, Model
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.layers.wrappers import Bidirectional
from keras.utils.data_utils import get_file
from keras.regularizers import WeightRegularizer
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from theano.ifelse import ifelse

Using Theano backend.
ERROR (theano.sandbox.cuda): Failed to compile cuda_ndarray.cu: libcublas.so.8.0: cannot open shared object file: No such file or directory


In [3]:
name = 'helpdesk'
args = {
    'inputdir': '../input/{}/'.format(name),   
    'outputdir': './output_files/{}/'.format(name)
}

args = argparse.Namespace(**args)

In [4]:
if not os.path.isdir(args.outputdir):
    os.makedirs(args.outputdir)

In [5]:
with open(args.inputdir + 'parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [6]:
with open(args.inputdir + 'preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

# Model

In [7]:
# build the model: 
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')

# train a 2-layer LSTM with one shared layer
l1 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=True, dropout_W=0.2)(main_input) # the shared layer
b1 = BatchNormalization()(l1)

l2_1 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2)(b1) # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)
l2_2 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2)(b1) # the layer specialized in time prediction
b2_2 = BatchNormalization()(l2_2)

act_output = Dense(len(targetchartoindice), activation='softmax', init='glorot_uniform', name='act_output')(b2_1)
time_output = Dense(1, init='glorot_uniform', name='time_output')(b2_2)

model = Model(input=[main_input], output=[act_output, time_output])

opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004, clipvalue=3)

model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint(args.outputdir + 'model_{epoch:02d}-{val_loss:.2f}.h5', 
                                   monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

model.fit(X, {'act_output':y_a, 'time_output':y_t}, validation_split=0.2, verbose=2, 
          callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, nb_epoch=500)

Build model...
Train on 7344 samples, validate on 1837 samples
Epoch 1/500
15s - loss: 2.1482 - act_output_loss: 0.9557 - time_output_loss: 1.1925 - val_loss: 1.7238 - val_act_output_loss: 0.6957 - val_time_output_loss: 1.0282
Epoch 2/500
15s - loss: 1.7612 - act_output_loss: 0.7136 - time_output_loss: 1.0476 - val_loss: 1.5787 - val_act_output_loss: 0.5862 - val_time_output_loss: 0.9925
Epoch 3/500
15s - loss: 1.7087 - act_output_loss: 0.6898 - time_output_loss: 1.0189 - val_loss: 1.5766 - val_act_output_loss: 0.5931 - val_time_output_loss: 0.9834
Epoch 4/500
15s - loss: 1.6867 - act_output_loss: 0.6771 - time_output_loss: 1.0096 - val_loss: 1.5426 - val_act_output_loss: 0.5759 - val_time_output_loss: 0.9667
Epoch 5/500
15s - loss: 1.6818 - act_output_loss: 0.6742 - time_output_loss: 1.0076 - val_loss: 1.5329 - val_act_output_loss: 0.5705 - val_time_output_loss: 0.9624
Epoch 6/500
15s - loss: 1.6693 - act_output_loss: 0.6661 - time_output_loss: 1.0032 - val_loss: 1.5562 - val_act_outp

15s - loss: 1.5933 - act_output_loss: 0.6101 - time_output_loss: 0.9832 - val_loss: 1.5309 - val_act_output_loss: 0.5710 - val_time_output_loss: 0.9599
Epoch 52/500
15s - loss: 1.6000 - act_output_loss: 0.6149 - time_output_loss: 0.9851 - val_loss: 1.5209 - val_act_output_loss: 0.5676 - val_time_output_loss: 0.9533
Epoch 53/500
15s - loss: 1.5974 - act_output_loss: 0.6134 - time_output_loss: 0.9840 - val_loss: 1.5178 - val_act_output_loss: 0.5654 - val_time_output_loss: 0.9524
Epoch 54/500
15s - loss: 1.5997 - act_output_loss: 0.6138 - time_output_loss: 0.9859 - val_loss: 1.5223 - val_act_output_loss: 0.5681 - val_time_output_loss: 0.9542
Epoch 55/500
15s - loss: 1.5945 - act_output_loss: 0.6108 - time_output_loss: 0.9836 - val_loss: 1.5191 - val_act_output_loss: 0.5665 - val_time_output_loss: 0.9526
Epoch 56/500
15s - loss: 1.5956 - act_output_loss: 0.6122 - time_output_loss: 0.9834 - val_loss: 1.5190 - val_act_output_loss: 0.5649 - val_time_output_loss: 0.9540
Epoch 57/500
15s - loss

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 15, 14)        0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 15, 100)       46000       main_input[0][0]                 
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 15, 100)       400         lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 100)           80400       batchnormalization_1[0][0]       
___________________________________________________________________________________________